### Relevant imports/variables.
These are mostly straightforward.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
# Some global constants.
NUM_PREDICTOR_COLS = 784
PREDICTOR_COLS = ['pixel' + str(i) for i in range(NUM_PREDICTOR_COLS)]

### Relevant helper routines
These are self explanatory.

In [3]:
def create_out_of_sample_score(given_pipeline, 
                               X_train, 
                               X_test, 
                               Y_train, 
                               Y_test):
    # Rewrite everything as a pipeline
    given_pipeline.fit(X_train, Y_train.values.ravel())
    predictions = given_pipeline.predict(X_test)
    out_of_sample_score = accuracy_score(predictions, Y_test)
    return (out_of_sample_score, predictions)

def cross_validate(my_pipeline, X, Y):
    cross_val_scores = \
        cross_val_score(my_pipeline, X, Y, scoring='accuracy', cv=5)

    return cross_val_scores.mean()

def train_test_cross_validate(train_data,
                              given_pipeline,
                              do_cross_validation=True,
                              X_columns=PREDICTOR_COLS, 
                              Y_columns=['label']):
    (X_train, X_test, Y_train, Y_test, X, Y) = \
        get_train_test_data(train_data, X_columns, Y_columns)
    out_of_sample_score, predictions_test = \
        create_out_of_sample_score(given_pipeline, X_train, X_test, Y_train, Y_test)

    predictions_train = given_pipeline.predict(X_train)
    num_correct_predictions_train = int((accuracy_score(predictions_train, Y_train)) * len(Y_train))
    num_correct_predictions_train1 = np.sum(predictions_train == Y_train.values.ravel())
    print('Training score is {0}'.format((accuracy_score(predictions_train, Y_train)) ))
    if do_cross_validation:
        cross_validation_score = cross_validate(given_pipeline, X, Y.values.ravel())
    else:
        cross_validation_score = -1

    return (out_of_sample_score, cross_validation_score)

def get_train_test_data(train_data,
                        X_columns=PREDICTOR_COLS, 
                        Y_columns=['label']):
    # Simple training and testing
    X = train_data[X_columns]
    Y = train_data[Y_columns]
 
    # Do imputation on relevant columns.
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)
    return (X_train, X_test, Y_train, Y_test, X, Y)



### Reading in data !

In [4]:
full_data = pd.read_csv('../input/train.csv')

### Checking for null values.

In [5]:
full_data.isnull().values.any()

False

### Split into training and validation sets

In [6]:
LEN_TRAIN_SET = int(0.8 * len(full_data))
train_data = full_data[0:LEN_TRAIN_SET]
validation_data = full_data[LEN_TRAIN_SET:len(full_data)]

In [7]:
assert(len(train_data) + len(validation_data) == len(full_data))

### Set up for doing cross validation
Later , we try several versions playing with the n_estimators parameter.

In [ ]:
# Commenting out as this takes a long time to complete now. Will revisit later.
#(out_of_sample_score, cross_validation_score) = \
#    train_test_cross_validate(full_data,
#                              make_pipeline(LogisticRegression()),
#                              do_cross_validation=False,
#                              X_columns=PREDICTOR_COLS, 
#                              Y_columns=['label'])
    
#print("Out of sample score is {0}\nCross val score is {1}".format(out_of_sample_score, cross_validation_score))
#Training score is 0.9443174603174603
#Out of sample score is 0.9054285714285715


### Check out PCA

In [8]:
from sklearn.decomposition import PCA
input_pca = PCA()
input_pca.fit(train_data[PREDICTOR_COLS])

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

### Determining number of components to use

To find out an optimal value, we calculate the amount of variance that is explained by including a specified number of components. For example, if we want to include only 10 components,  we select the top 10 of them by variability and see how much of variance, is explained by all of these together.

We plan to plot of the amount of variability explained versus the number of components include and gauge an optimal value from the same.

#### Create a data array

In [9]:
X = np.arange(1,101)
Y = np.zeros(100)
for i in range(100):
    Y[i] = input_pca.explained_variance_ratio_[0:(i+1)].sum()

#### Plot data 

In [10]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1,figsize=(16, 9)) 
ax.set_title('Cum Variability vs Number of PCA components')
ax.set_xlabel('Number of PCA components')
ax.set_ylabel('Cumulative variability explained')
ax.set_yticks(np.arange(0, 0.9, 0.05))
ax.set_xticks(np.arange(0, 100, 5))
ax.plot(X, Y)


#### Comment

Since, we see diminising returns here, we decide to include 50 principal components (as that explains 80% of the variability) and the marginal improvement in variability in addition of an extra components looks very small.

### Cross validation and out of sample testing

In [ ]:
# Commenting out as this takes a long time to complete now. Will revisit later.
from sklearn.pipeline import Pipeline
my_pipeline = Pipeline([('pca', PCA(n_components=50)), ('logistic', LogisticRegression())])
(out_of_sample_score, cross_validation_score) = \
    train_test_cross_validate(full_data,
                              my_pipeline,
                              do_cross_validation=False,
                              X_columns=PREDICTOR_COLS, 
                              Y_columns=['label'])
    
print("Out of sample score is {0}\nCross val score is {1}".format(out_of_sample_score, cross_validation_score))
#Training score is 0.9443174603174603
#Out of sample score is 0.9054285714285715


### Making predictions on  test data.

In [ ]:
test_data = pd.read_csv('../input/test.csv')
test_predictions = my_pipeline.predict(test_data[PREDICTOR_COLS])
test_data['label'] = test_predictions
test_data['ImageId'] = test_data.index + 1
test_data[['ImageId', 'label']].to_csv('submission_randomforest_sklearn.csv', index=False)